In [7]:
from keras.models import model_from_json
from keras.optimizers import SGD
from time import sleep
from scipy.ndimage import zoom
import numpy as np
import cv2
import datetime
import time


In [8]:
# function to handle current time at prediction
def current_time():
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
    return(st)

In [9]:
model = model_from_json(open('models/Face_model_architecture.json').read())
#model.load_weights('_model_weights.h5')
model.load_weights('models/Face_model_weights.h5')
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

In [10]:
def extract_face_features(gray, detected_face, offset_coefficients):
        (x, y, w, h) = detected_face
        #print x , y, w ,h
        horizontal_offset = np.int(np.floor(offset_coefficients[0] * w))
        vertical_offset = np.int(np.floor(offset_coefficients[1] * h))

        extracted_face = gray[y+vertical_offset:y+h, 
                          x+horizontal_offset:x-horizontal_offset+w]
        #print extracted_face.shape
        new_extracted_face = zoom(extracted_face, (48. / extracted_face.shape[0], 
                                               48. / extracted_face.shape[1]))
        new_extracted_face = new_extracted_face.astype(np.float32)
        new_extracted_face /= float(new_extracted_face.max())
        return new_extracted_face

In [ ]:
def detect_face(frame):
        cascPath = "models/haarcascade_frontalface_default.xml"
        faceCascade = cv2.CascadeClassifier(cascPath)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        detected_faces = faceCascade.detectMultiScale(
                gray,
                scaleFactor=1.1,
                minNeighbors=6,
                minSize=(48, 48),
                flags=cv2.FONT_HERSHEY_SIMPLEX
            )
        return gray, detected_faces

In [ ]:
#cascPath = "haarcascade_frontalface_default.xml"
#faceCascade = cv2.CascadeClassifier(cascPath)

video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    sleep(0.2)
    ret, frame = video_capture.read()

    # detect faces
    gray, detected_faces = detect_face(frame)
    
    face_index = 0
    
    # predict output
    for face in detected_faces:
        (x, y, w, h) = face
        if w > 10:
            # draw rectangle around face 
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # extract features
            extracted_face = extract_face_features(gray, face, (0.075, 0.05)) #(0.075, 0.05)

            # predict smile
            prediction_result = model.predict_classes(extracted_face.reshape(1,48,48,1))

            # draw extracted face in the top right corner
            #frame[face_index * 48: (face_index + 1) * 48, -49:-1, :] = cv2.cvtColor(extracted_face * 255, cv2.COLOR_GRAY2RGB)

            # annotate main image with a label
            if prediction_result == 0 :
                cv2.putText(frame, "Angry", (x,y),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Angry ||", current_time())
            
            elif prediction_result == 1:
                cv2.putText(frame, "Disgust", (x,y) ,cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Disgust ||", current_time())

            elif prediction_result == 2:
                cv2.putText(frame, "Fear", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Fear ||", current_time())

            elif prediction_result == 3:
                cv2.putText(frame, "Happy", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Happy ||", current_time())

            elif prediction_result == 4:
                cv2.putText(frame, "Sad", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Sad ||", current_time())

            elif prediction_result == 5:
                cv2.putText(frame, "Surprise",(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Surprise ||", current_time())

            else:
                cv2.putText(frame, "Neutral", (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("Prediction : Neutral ||",  current_time())


            # increment counter
            face_index += 1
                

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

/home/jugal/.virtualenvs/cv/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Prediction : Sad || 22:59:09
Prediction : Sad || 22:59:09
Prediction : Happy || 22:59:09
Prediction : Fear || 22:59:09
Prediction : Fear || 22:59:10
Prediction : Sad || 22:59:10
Prediction : Sad || 22:59:10
Prediction : Neutral || 22:59:11
Prediction : Sad || 22:59:11
Prediction : Neutral || 22:59:11
Prediction : Sad || 22:59:11
Prediction : Sad || 22:59:12
Prediction : Neutral || 22:59:12
Prediction : Sad || 22:59:12
Prediction : Neutral || 22:59:13


In [ ]:
import os
os._exit(0)